In [1]:
# conda install -c conda-forge pyspark 
# import findspark
# findspark.init()
from sedona.spark import SedonaContext
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, LongType, ArrayType
from pyspark.sql import Row
from pprint import pprint
from time import sleep
from pyspark.sql.window import Window
import pandas as pd
# import Transformer
import geopandas as gpd
from pyproj import Transformer
pd.set_option('display.max_columns', None)
import os
import re
from sklearn.cluster import DBSCAN
import geopandas as gpd
from pyspark.sql.functions import col, udf
!pip install clickhouse_connect
!pip install pyxlsb
!pip install osmium shapely
!pip install python-dotenv
import clickhouse_connect
from pyspark.sql.functions import split, col
from pyspark.sql.functions import sum, avg, count
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import os
from dotenv import load_dotenv
from pyspark.sql import Window
from pyspark.sql.functions import row_number, rank, dense_rank
from pyspark.sql.functions import col, split, when, last, monotonically_increasing_id, expr 
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, StringType
import pyspark.sql.functions as F
import builtins 
from collections import Counter
from pyspark.sql.functions import to_date, year, month, dayofmonth
import geopandas as gpd
import os
from glob import glob

import numpy as np
import pandas as pd
from PIL import Image
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import osmnx as ox
import geopandas as gpd

load_dotenv('../.env')

CH_IP = os.getenv('CH_IP')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')

client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD)

packages = [
        "com.clickhouse.spark:clickhouse-spark-runtime-3.5_2.12:0.8.0"
        ,"com.clickhouse:clickhouse-jdbc:0.7.0"
        ,"com.clickhouse:clickhouse-client:0.7.0"
        ,"com.clickhouse:clickhouse-http-client:0.7.0"
        ,"org.apache.httpcomponents.client5:httpclient5:5.3.1"
        ,'org.apache.sedona:sedona-spark-3.5_2.12:1.7.0'
        ,'org.datasyslab:geotools-wrapper:1.7.0-28.5'
        ,'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'
    ]

ram = 80
cpu = 4*3

spark = (
    SparkSession.builder
        .appName('appName')
        .master(f"spark://127.0.0.1:7077")
        .config("spark.jars.packages", ','.join(packages))
        .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all')
        .config("spark.sql.catalog.clickhouse", "com.clickhouse.spark.ClickHouseCatalog")
        .config("spark.sql.catalog.clickhouse.host", f"{CH_IP}")
        .config("spark.sql.catalog.clickhouse.protocol", "http")
        .config("spark.sql.catalog.clickhouse.http_port", "8123")
        .config("spark.sql.catalog.clickhouse.user", f"{CLICKHOUSE_USER}")
        .config("spark.sql.catalog.clickhouse.password", f"{CLICKHOUSE_PASSWORD}")
        .config("spark.sql.catalog.clickhouse.database", "default")
        .config("spark.executor.memory", f"{ram}g")
        .config("spark.driver.memory", f"{ram}g")
        .config("spark.driver.maxResultSize", f"{ram}g")
        .config("spark.executor.memoryOverhead", f"{ram}g")
        .config("spark.clickhouse.write.format", "json")
        .getOrCreate()
    )

#sedona = SedonaContext.create(spark) 
SedonaRegistrator.registerAll(spark)
spark.sql("use clickhouse")
base = '/home/wattwillersci'
path_data = f"{base}/data"


https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/wattwillersci/.ivy2/cache
The jars for the packages stored in: /home/wattwillersci/.ivy2/jars
com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-client added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
uk.co.gresearch.spark#spark-extension_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-26d0790e-490d-4eaa-a5d7-60d0674dda52;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/conda/envs/sp/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.clickhouse.spark#clickhouse-spark-runtime-3.5_2.12;0.8.0 in central
	found com.clickhouse#clickhouse-jdbc;0.7.0 in central
	found com.clickhouse#clickhouse-client;0.7.0 in central
	found com.clickhouse#clickhouse-data;0.7.0 in central
	found com.clickhouse#clickhouse-http-client;0.7.0 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.4 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.0 in central
	found org.apache.sedona#sedona-common;1.7.0 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in centr

### Cameras with location in NYC

In [2]:
nyc_cameras = spark.read.csv(f"{path_data}/nyc_cameras.csv", header=True, inferSchema=True)
nyc_cameras.show(5)
nyc_cameras.createOrReplaceTempView("nyc_cameras")

+---------+----------+
| latitude| longitude|
+---------+----------+
|40.710983|-73.963168|
|40.675787|-73.896898|
|40.715157| -74.00213|
|40.761186|-73.955676|
|  40.6332|  -74.0057|
+---------+----------+
only showing top 5 rows



In [3]:
nyc_districts = (
    spark.read.format("geojson").option("multiLine", "true").load(f"{path_data}/nyc_districts.json")
    .selectExpr("explode(features) as features")
    .select("features.*")
    .withColumn("OBJECTID", expr("properties['OBJECTID']"))
    .withColumn("BoroCD", expr("properties['BoroCD']"))
    .drop("properties")
    .drop("type")
    .drop("id")
)
nyc_districts.show(truncate=True)
nyc_districts.createOrReplaceTempView("nyc_districts")

+--------------------+--------+------+
|            geometry|OBJECTID|BoroCD|
+--------------------+--------+------+
|POLYGON ((-73.970...|       1|   308|
|POLYGON ((-73.801...|       2|   410|
|POLYGON ((-73.881...|       3|   206|
|POLYGON ((-73.867...|       4|   480|
|POLYGON ((-73.984...|       5|   104|
|POLYGON ((-73.980...|       6|   107|
|POLYGON ((-74.193...|       7|   503|
|MULTIPOLYGON (((-...|       8|   595|
|POLYGON ((-73.877...|       9|   207|
|POLYGON ((-73.925...|      10|   317|
|POLYGON ((-73.840...|      11|   481|
|POLYGON ((-73.872...|      12|   227|
|POLYGON ((-73.820...|      13|   409|
|POLYGON ((-73.903...|      14|   316|
|POLYGON ((-73.728...|      15|   413|
|POLYGON ((-73.799...|      16|   483|
|POLYGON ((-73.895...|      17|   208|
|POLYGON ((-73.878...|      18|   226|
|POLYGON ((-73.991...|      19|   312|
|POLYGON ((-73.959...|      20|   314|
+--------------------+--------+------+
only showing top 20 rows



In [4]:
cameras = spark.sql("""
    with crimes as (
                    select *
            ,ST_Point(Longitude , Latitude) as geom
        from nyc_cameras
        where Latitude is not null or Longitude is not null
    ),
    districs as (
    select 
        OBJECTID
        ,BoroCD as district
        ,geometry as geom
    from nyc_districts
    )
                      select
                        crimes.*
                      , districs.district
                        , ST_Intersects(crimes.geom, districs.geom) as intersects
                      from crimes
                      join districs on ST_Intersects(crimes.geom, districs.geom)
""")

cameras.write.mode('overwrite').parquet(f"{path_data}/cameras.parquet")
cameras.show(5)

+----------------+-----------------+--------------------+--------+----------+
|        latitude|        longitude|                geom|district|intersects|
+----------------+-----------------+--------------------+--------+----------+
|40.6809634749315|-73.9670419692993|POINT (-73.967041...|     308|      true|
|       40.659301|       -73.854457|POINT (-73.854457...|     410|      true|
|       40.662511|       -73.840815|POINT (-73.840815...|     410|      true|
|       40.665007|       -73.819358|POINT (-73.819358...|     410|      true|
|         40.6666|         -73.8104|POINT (-73.8104 4...|     410|      true|
+----------------+-----------------+--------------------+--------+----------+
only showing top 5 rows



In [ ]:
disricts_counts = cameras.groupBy("district").agg(
    F.count("*").alias("cameras_in_district")).orderBy("cameras_in_district", ascending=False)
disricts_counts.show()

+--------+-------------------+
|district|cameras_in_district|
+--------+-------------------+
|     302|                 68|
|     105|                 57|
|     103|                 52|
|     101|                 46|
|     402|                 41|
|     108|                 40|
|     502|                 39|
|     503|                 33|
|     104|                 32|
|     106|                 32|
|     501|                 27|
|     301|                 27|
|     306|                 25|
|     413|                 24|
|     307|                 23|
|     407|                 21|
|     102|                 19|
|     411|                 17|
|     112|                 16|
|     107|                 16|
+--------+-------------------+
only showing top 20 rows



In [5]:
districts_counts = cameras.groupBy("district") \
    .agg(F.count("*").alias("cameras_in_district"))

cameras = cameras.join(districts_counts, on="district", how="left")
cameras.write.mode("overwrite").parquet(f"{path_data}/cameras.parquet")

In [37]:
cameras.show(5)

+--------+----------------+-----------------+--------------------+----------+-------------------+
|district|        latitude|        longitude|                geom|intersects|cameras_in_district|
+--------+----------------+-----------------+--------------------+----------+-------------------+
|     308|40.6809634749315|-73.9670419692993|POINT (-73.967041...|      true|                  1|
|     410|       40.659301|       -73.854457|POINT (-73.854457...|      true|                  9|
|     410|       40.662511|       -73.840815|POINT (-73.840815...|      true|                  9|
|     410|       40.665007|       -73.819358|POINT (-73.819358...|      true|                  9|
|     410|         40.6666|         -73.8104|POINT (-73.8104 4...|      true|                  9|
+--------+----------------+-----------------+--------------------+----------+-------------------+
only showing top 5 rows



In [2]:
cameras = spark.read.parquet(f"{path_data}/cameras.parquet")

### Colorfull png data

In [3]:
nyc_districts = gpd.read_file(f"{path_data}/nyc_districts.json")
dist =  nyc_districts.to_crs("EPSG:4326")

In [16]:
dist

,OBJECTID,BoroCD,Shape__Area,Shape__Length,geometry
0,1,308,4.560379e+07,38232.886995,"POLYGON ((-73.97083 40.68162, -73.96768 40.680..."
1,2,410,1.785196e+08,68165.986977,"POLYGON ((-73.80116 40.67181, -73.80114 40.671..."
2,3,206,4.266432e+07,35875.709011,"POLYGON ((-73.8812 40.86169, -73.88056 40.8612..."
3,4,480,5.326668e+07,36729.979587,"POLYGON ((-73.86789 40.78737, -73.86803 40.787..."
4,5,104,7.311657e+07,39049.714125,"POLYGON ((-73.98499 40.76933, -73.98477 40.769..."
...,...,...,...,...,...
66,67,502,1.044529e+09,244755.942909,"MULTIPOLYGON (((-74.10174 40.55991, -74.10237 ..."
67,68,309,4.531564e+07,29934.144533,"POLYGON ((-73.94508 40.65573, -73.94716 40.655..."
68,69,355,2.614351e+07,23963.455774,"POLYGON ((-73.96888 40.67299, -73.96894 40.672..."
69,70,101,1.096508e+08,67287.276711,"MULTIPOLYGON (((-74.04388 40.6902, -74.04351 4..."


In [9]:
#!pip install rasterio

In [5]:
from PIL import Image
import numpy as np
from rasterio.transform import from_bounds

img = Image.open(f"{path_data}/photos/race_white.png").convert("RGB")
arr = np.array(img)
h, w, _ = arr.shape

west, south, east, north = -74.25559, 40.49612, -73.70001, 40.91553
transform = from_bounds(west, south, east, north, w, h)


In [6]:
import rasterio
from rasterio.features import rasterize

shapes = ((geom, idx) for idx, geom in enumerate(dist.geometry))

mask = rasterize(
    shapes=shapes,
    out_shape=(h, w),
    transform=transform,
    fill=-1,
    dtype="int32"
)


In [ ]:
import pandas as pd

results = []
for idx, row in dist.iterrows():
    pix = arr[mask == idx]
    if pix.size == 0:
        continue
    avg_r, avg_g, avg_b = pix.mean(axis=0)
    warmth = float(avg_r) - float(avg_b)
    results.append({
        "District":      row["BoroCD"],    
        "avg_r":        avg_r,
        "avg_g":        avg_g,
        "avg_b":        avg_b,
        "warmth":       warmth
    })

df = pd.DataFrame(results)
df.to_csv("race_white_by_cd.csv", index=False)


In [18]:
df= pd.read_csv("race_white_by_cd.csv")
df.head()

,District,avg_r,avg_g,avg_b,warmth
0,308,136.825104,244.247784,154.640351,-17.815247
1,410,113.921029,223.003579,157.093703,-43.172674
2,206,121.001542,245.620457,141.730647,-20.729104
3,480,146.184361,217.219209,188.936022,-42.751661
4,104,118.913381,206.551588,143.278892,-24.365510


In [7]:
photos_dir = os.path.join(path_data, "photos")
output_csv = os.path.join(path_data, "photos_by_district.csv")
west, south, east, north = -74.25559, 40.49612, -73.70001, 40.91553


def process_image(fname, dist, transform):
    img = Image.open(fname).convert("RGB")
    arr = np.array(img)
    h, w, _ = arr.shape

    mask = rasterize(
        ((geom, i) for i, geom in enumerate(dist.geometry)),
        out_shape=(h, w),
        transform=transform,
        fill=-1,
        dtype="int32"
    )

    rows = []
    for i, row in dist.iterrows():
        pix = arr[mask == i]
        if pix.size == 0:
            continue
        avg_r, avg_g, avg_b = pix.mean(axis=0)
        warmth = avg_r - avg_b
        rows.append({
            "District": row["BoroCD"],
            "avg_r":    float(avg_r),
            "avg_g":    float(avg_g),
            "avg_b":    float(avg_b),
            "warmth":   float(warmth),
            "image":    os.path.splitext(os.path.basename(fname))[0],
        })
    return pd.DataFrame(rows)

png_files = sorted(glob(os.path.join(photos_dir, "*.png")))

all_dfs   = []
transform = None

for png in png_files:
    img = Image.open(png)
    w, h = img.size
    if transform is None:
        transform = from_bounds(west, south, east, north, w, h)
    df_img = process_image(png, dist, transform)
    all_dfs.append(df_img)

final = pd.concat(all_dfs, ignore_index=True)
final.to_csv(output_csv, index=False)
print(f"Wrote {len(final)} rows to {output_csv!r}")

Wrote 426 rows to '/home/wattwillersci/data/photos_by_district.csv'


In [ ]:
result_df = pd.DataFrame({"District": dist["BoroCD"]})

transform = None

for png_path in sorted(glob(os.path.join(photos_dir, "*.png"))):
    name = os.path.splitext(os.path.basename(png_path))[0]

    img = Image.open(png_path).convert("RGB")
    arr = np.array(img)
    h, w = arr.shape[:2]

    if transform is None:
        transform = from_bounds(west, south, east, north, w, h)

    mask = rasterize(
        ((geom, i) for i, geom in enumerate(dist.geometry)),
        out_shape=(h, w),
        transform=transform,
        fill=-1,
        dtype="int32"
    )

    data = {
        "District":            [],
        f"R_avg_{name}":       [],
        f"G_avg_{name}":       [],
        f"B_avg_{name}":       [],
        f"warmth_{name}":      [],
    }

    for i, row in dist.iterrows():
        pix = arr[mask == i]
        if pix.size == 0:
            avg_r = avg_g = avg_b = np.nan
        else:
            avg_r, avg_g, avg_b = pix.mean(axis=0)
        warmth = avg_r - avg_b if pix.size > 0 else np.nan

        data["District"].append(row["BoroCD"])
        data[f"R_avg_{name}"].append(float(avg_r))
        data[f"G_avg_{name}"].append(float(avg_g))
        data[f"B_avg_{name}"].append(float(avg_b))
        data[f"warmth_{name}"].append(float(warmth))

    df_img = pd.DataFrame(data)
    result_df = result_df.merge(df_img, on="District", how="left")


result_df.to_csv(output_csv, index=False)
print(f"Готово — {len(result_df)} строк, файл: {output_csv!r}")

Готово — 71 строк, файл: '/home/wattwillersci/data/photos_by_district.csv'


In [9]:
df = pd.read_csv(output_csv)
df.head()

,District,R_avg_education_collegeor_above,G_avg_education_collegeor_above,B_avg_education_collegeor_above,warmth_education_collegeor_above,R_avg_family_size,G_avg_family_size,B_avg_family_size,warmth_family_size,R_avg_home_value,G_avg_home_value,B_avg_home_value,warmth_home_value,R_avg_income_household_med,G_avg_income_household_med,B_avg_income_household_med,warmth_income_household_med,R_avg_race_black,G_avg_race_black,B_avg_race_black,warmth_race_black,R_avg_race_white,G_avg_race_white,B_avg_race_white,warmth_race_white
0,308,228.739374,119.007506,122.233225,106.506149,237.262073,234.534364,93.875475,143.386598,112.858474,226.226081,131.042413,-18.183939,236.716404,103.790016,108.325375,128.391029,86.076415,86.229788,85.970338,0.106077,136.825104,244.247784,154.640351,-17.815247
1,410,209.020389,148.982289,167.503013,41.517376,183.590043,195.647355,189.069504,-5.479461,170.535733,206.952871,203.440760,-32.905027,190.043480,173.809684,191.370979,-1.327499,101.693098,112.262036,128.691620,-26.998522,113.921029,223.003579,157.093703,-43.172674
2,206,234.648125,132.571484,132.848067,101.800058,230.483081,231.851441,88.258459,142.224622,142.832064,244.947267,160.349465,-17.517401,234.648125,132.571484,132.848067,101.800058,87.219898,89.924130,87.536971,-0.317073,121.001542,245.620457,141.730647,-20.729104
3,480,202.227245,159.588626,182.537166,19.690079,188.456266,199.595889,170.073636,18.382630,146.628960,210.495055,187.992119,-41.363159,202.239839,155.095117,178.344383,23.895457,152.778087,144.432700,166.104698,-13.326611,146.184361,217.219209,188.936022,-42.751661
4,104,180.302658,117.133363,126.508898,53.793760,232.383251,235.587125,169.239468,63.143782,135.385672,198.783453,156.084310,-20.698637,188.737610,116.681967,126.515319,62.222291,137.313359,121.434839,129.344222,7.969137,118.913381,206.551588,143.278892,-24.365510


In [6]:
colors = spark.read.csv(f"{path_data}/photos_by_district.csv", header=True, inferSchema=True)
colors.show(5)

+--------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------------+--------------------------+--------------------------+---------------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-------------------+
|District|R_avg_education_collegeor_above|G_avg_education_collegeor_above|B_avg_education_collegeor_above|warmth_education_collegeor_above| R_avg_family_size| G_avg_family_size| B_avg_family_size|warmth_family_size|  R_avg_home_value|  G_avg_home_value|  B_avg_home_value|  warmth_home_value|R_avg_income_household_med|G_avg_income_household_med|B_avg_income_household_med|warmth_income_household_med|  R_avg_race_black|  G_avg_race_black|  B_

In [4]:
cameras = spark.read.parquet(f"{path_data}/cameras.parquet")

In [5]:
cameras = cameras.drop("latitude", "longitude", "geom", "intersects" )

In [7]:
joined = colors.join(cameras, on="district", how="left")
joined.show(5)

25/07/29 19:23:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------------+--------------------------+--------------------------+---------------------------+------------------+------------------+-----------------+-------------------+------------------+-----------------+-----------------+-------------------+-------------------+
|District|R_avg_education_collegeor_above|G_avg_education_collegeor_above|B_avg_education_collegeor_above|warmth_education_collegeor_above| R_avg_family_size| G_avg_family_size| B_avg_family_size|warmth_family_size|  R_avg_home_value|  G_avg_home_value|  B_avg_home_value|  warmth_home_value|R_avg_income_household_med|G_avg_income_household_med|B_avg_income_household_med|warmth_income_household_med|  R_avg_race_black|  G_avg

In [8]:
joined.write.mode("overwrite").parquet(f"{path_data}/joined.parquet")

In [9]:
previous = spark.read.parquet(f"{path_data}/version_for_shap.parquet")
previous.show(5)

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|district|CRIME_YEAR|CRIME_MONTH|            R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_median|G_median|B_median|R_q25|G_q25|B_q25|R_q75|G_q75|B_q75|    brightness_avg|   dark_pixel_ratio|BORO_NM|     SUSP_RACE|SUSP_SEX|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|CRIME_DAY|total_crim

In [10]:
merged = previous.join(joined, on="district", how="left")
merged.show(5)

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+-------+---------+--------+-------------+--------------+-------+---------+-----------------+------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------------+--------------------------+---------

In [11]:
merged.write.mode("overwrite").parquet(f"{path_data}/crime_nyc_years_shap.parquet")

In [12]:
merged.select("district").distinct().count()

42

In [15]:
df = spark.read.parquet(f"{path_data}/crime_nyc_years_shap.parquet")
df.show(5)

+--------+----------+-----------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------------+-----------------------

In [16]:
df = df.withColumn("quarter", F.ceil(F.col("CRIME_MONTH")/3))


In [18]:
df.show(5)

+--------+----------+-----------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+--------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+-------------------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------------+-----------------------

In [ ]:
df.write.mode("overwrite").parquet(f"{path_data}/crime_nyc_years_shap_quat.parquet")

In [3]:
df.select("district", "CRIME_YEAR", "CRIME_MONTH").distinct().show()

+--------+----------+-----------+
|district|CRIME_YEAR|CRIME_MONTH|
+--------+----------+-----------+
|     403|      2022|          4|
|     403|      2024|          3|
|     306|      2022|          3|
|     209|      2023|         12|
|     404|      2022|          3|
|     306|      2024|          2|
|     205|      2024|          6|
|     304|      2025|          3|
|     304|      2023|          8|
|     210|      2025|          1|
|     309|      2024|          4|
|     210|      2022|          6|
|     307|      2021|          9|
|     501|      2022|          1|
|     403|      1999|          1|
|     210|      2023|          8|
|     210|      2024|          7|
|     205|      2024|          7|
|     301|      2022|          6|
|     403|      2021|          9|
+--------+----------+-----------+
only showing top 20 rows



In [7]:
df2 = df.withColumn(
    "quarter",
    F.ceil(F.col("CRIME_MONTH") / 3)
)
df2.select("CRIME_YEAR", "CRIME_MONTH", "quarter").show()

+----------+-----------+-------+
|CRIME_YEAR|CRIME_MONTH|quarter|
+----------+-----------+-------+
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          1|      1|
|      2025|          1|      1|
|      2025|          2|      1|
|      2025|          2|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          2|      1|
|      2025|          2|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          3|      1|
|      2025|          1|      1|
|      2025|          1|      1|
|      2025|          1|      1|
|      2025|          1|      1|
+----------+-----------+-------+
only showing top 20 rows



In [ ]:
# result = df.groupBy("district", "CRIME_YEAR") \
#     .agg(
#         F.countDistinct("CRIME_MONTH").alias("num_distinct_months"),
#         F.sort_array(F.collect_set("CRIME_MONTH")).alias("months_list")
#     )

# result = result.filter(F.col("num_distinct_months") > 4)

# result.show()

+--------+----------+-------------------+--------------------+
|district|CRIME_YEAR|num_distinct_months|         months_list|
+--------+----------+-------------------+--------------------+
|     201|      2024|                 12|[1, 2, 3, 4, 5, 6...|
|     202|      2024|                 11|[1, 3, 4, 5, 6, 7...|
|     203|      2024|                 11|[1, 2, 3, 4, 5, 6...|
|     204|      2024|                 12|[1, 2, 3, 4, 5, 6...|
|     205|      2023|                  6| [1, 2, 3, 5, 7, 12]|
|     205|      2024|                 11|[1, 2, 3, 5, 6, 7...|
|     206|      2024|                 10|[1, 4, 5, 6, 7, 8...|
|     207|      2024|                 11|[1, 2, 3, 5, 6, 7...|
|     208|      2024|                 11|[1, 2, 3, 4, 6, 7...|
|     209|      2023|                  5|  [7, 8, 10, 11, 12]|
|     209|      2024|                 12|[1, 2, 3, 4, 5, 6...|
|     210|      2024|                 11|[1, 2, 3, 5, 6, 7...|
|     211|      2024|                 12|[1, 2, 3, 4, 5

In [14]:
result.select("CRIME_YEAR").distinct().count()

4

In [15]:
valid_months = result.select(
    "district",
    "CRIME_YEAR",
    F.explode("months_list").alias("CRIME_MONTH")
)

df_filtered = (
    df
    .join(valid_months,
          on=["district", "CRIME_YEAR", "CRIME_MONTH"],
          how="inner")
)

In [16]:
df_filtered.toPandas()

,district,CRIME_YEAR,CRIME_MONTH,R_avg,G_avg,B_avg,R_stddev,G_stddev,B_stddev,R_median,G_median,B_median,R_q25,G_q25,B_q25,R_q75,G_q75,B_q75,brightness_avg,dark_pixel_ratio,BORO_NM,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CRIME_DAY,total_crimes_dist,mean_white,mean_black,mean_hispanic,mean_amer_indian,mean_asian,mean_M,mean_F,mean_U,pct_age_18_24,pct_age_25_44,pct_age_45_64,pct_age_<18,pct_age_65_plus,pct_violent_crimes,pct_property_crimes,pct_other_offenses,R_avg_education_collegeor_above,G_avg_education_collegeor_above,B_avg_education_collegeor_above,warmth_education_collegeor_above,R_avg_family_size,G_avg_family_size,B_avg_family_size,warmth_family_size,R_avg_home_value,G_avg_home_value,B_avg_home_value,warmth_home_value,R_avg_income_household_med,G_avg_income_household_med,B_avg_income_household_med,warmth_income_household_med,R_avg_race_black,G_avg_race_black,B_avg_race_black,warmth_race_black,R_avg_race_white,G_avg_race_white,B_avg_race_white,warmth_race_white,cameras_in_district
0,209,2024,12,172.182071,143.048645,87.474983,52.852070,62.837106,84.444922,165,128,52,127,93,27,215,181,117,134.235233,0.014767,BRONX,BLACK,M,45-64,WHITE HISPANIC,M,31,56,0.035714,0.285714,0.517857,0.017857,0.053571,0.303571,0.142857,0.375000,0.007653,0.010714,0.035714,0.0,0.0,0.142857,0.571429,0.285714,225.428567,120.998728,131.346440,94.082127,225.949197,229.384702,103.930203,122.018994,135.802151,237.566677,164.614511,-28.812360,225.396304,109.741381,121.045774,104.35053,99.864859,96.979552,104.159861,-4.295002,105.130718,238.541018,136.882036,-31.751318,2
1,209,2024,12,172.182071,143.048645,87.474983,52.852070,62.837106,84.444922,165,128,52,127,93,27,215,181,117,134.235233,0.014767,BRONX,BLACK,M,45-64,WHITE HISPANIC,M,31,56,0.035714,0.285714,0.517857,0.017857,0.053571,0.303571,0.142857,0.375000,0.007653,0.010714,0.035714,0.0,0.0,0.142857,0.571429,0.285714,225.428567,120.998728,131.346440,94.082127,225.949197,229.384702,103.930203,122.018994,135.802151,237.566677,164.614511,-28.812360,225.396304,109.741381,121.045774,104.35053,99.864859,96.979552,104.159861,-4.295002,105.130718,238.541018,136.882036,-31.751318,2
2,209,2024,7,172.182071,143.048645,87.474983,52.852070,62.837106,84.444922,165,128,52,127,93,27,215,181,117,134.235233,0.014767,BRONX,WHITE HISPANIC,M,<18,WHITE HISPANIC,M,1,6,0.000000,0.000000,0.333333,0.000000,0.166667,0.833333,0.000000,0.166667,0.023810,0.016667,0.333333,0.0,0.0,0.166667,0.833333,0.000000,225.428567,120.998728,131.346440,94.082127,225.949197,229.384702,103.930203,122.018994,135.802151,237.566677,164.614511,-28.812360,225.396304,109.741381,121.045774,104.35053,99.864859,96.979552,104.159861,-4.295002,105.130718,238.541018,136.882036,-31.751318,2
3,209,2024,7,172.182071,143.048645,87.474983,52.852070,62.837106,84.444922,165,128,52,127,93,27,215,181,117,134.235233,0.014767,BRONX,WHITE HISPANIC,M,<18,WHITE HISPANIC,M,1,6,0.000000,0.000000,0.333333,0.000000,0.166667,0.833333,0.000000,0.166667,0.023810,0.016667,0.333333,0.0,0.0,0.166667,0.833333,0.000000,225.428567,120.998728,131.346440,94.082127,225.949197,229.384702,103.930203,122.018994,135.802151,237.566677,164.614511,-28.812360,225.396304,109.741381,121.045774,104.35053,99.864859,96.979552,104.159861,-4.295002,105.130718,238.541018,136.882036,-31.751318,2
4,209,2024,12,172.182071,143.048645,87.474983,52.852070,62.837106,84.444922,165,128,52,127,93,27,215,181,117,134.235233,0.014767,BRONX,UNKNOWN,U,25-44,WHITE HISPANIC,M,31,56,0.035714,0.285714,0.517857,0.017857,0.053571,0.303571,0.142857,0.375000,0.007653,0.010714,0.035714,0.0,0.0,0.142857,0.571429,0.285714,225.428567,120.998728,131.346440,94.082127,225.949197,229.384702,103.930203,122.018994,135.802151,237.566677,164.614511,-28.812360,225.396304,109.741381,121.045774,104.35053,99.864859,96.979552,104.159861,-4.295002,105.130718,238.541018,136.882036,-31.751318,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [17]:
base = '/home/wattwillersci'
path_data = f"{base}/data"
df_filtered.write.mode("overwrite").parquet(f"{path_data}/crime_nyc_years_shap1")

### Working with multiple years dataset

In [2]:
nyc_crime = spark.read.csv(f"{path_data}/NYPD_currentyear.csv", header=True, inferSchema=True)
nyc_crime.show(5)

25/07/20 17:00:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----------+-------------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+----------------+-----+-----------+-----------------+--------------------+--------+--------------------+-----+--------------------+--------------------+----------+------------+--------------+---------+--------+----------------+-------------+--------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+
|CMPLNT_NUM|ADDR_PCT_CD|      BORO_NM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|CRM_ATPT_CPTD_CD|HADEVELOPT|HOUSING_PSA|JURISDICTION_CODE|      JURIS_DESC|KY_CD| LAW_CAT_CD|LOC_OF_OCCUR_DESC|           OFNS_DESC|PARKS_NM|         PATROL_BORO|PD_CD|             PD_DESC|       PREM_TYP_DESC|    RPT_DT|STATION_NAME|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|X_COORD_CD|Y_COORD_CD|        Latitude|        Longitude|             

In [7]:
nyc_crime.createOrReplaceTempView("nyc_crime_years")

In [8]:
nyc_districts = (
    spark.read.format("geojson").option("multiLine", "true").load(f"{path_data}/nyc_districts.json")
    .selectExpr("explode(features) as features")
    .select("features.*")
    .withColumn("OBJECTID", expr("properties['OBJECTID']"))
    .withColumn("BoroCD", expr("properties['BoroCD']"))
    .drop("properties")
    .drop("type")
    .drop("id")
)
nyc_districts.show(truncate=True)

+--------------------+--------+------+
|            geometry|OBJECTID|BoroCD|
+--------------------+--------+------+
|POLYGON ((-73.970...|       1|   308|
|POLYGON ((-73.801...|       2|   410|
|POLYGON ((-73.881...|       3|   206|
|POLYGON ((-73.867...|       4|   480|
|POLYGON ((-73.984...|       5|   104|
|POLYGON ((-73.980...|       6|   107|
|POLYGON ((-74.193...|       7|   503|
|MULTIPOLYGON (((-...|       8|   595|
|POLYGON ((-73.877...|       9|   207|
|POLYGON ((-73.925...|      10|   317|
|POLYGON ((-73.840...|      11|   481|
|POLYGON ((-73.872...|      12|   227|
|POLYGON ((-73.820...|      13|   409|
|POLYGON ((-73.903...|      14|   316|
|POLYGON ((-73.728...|      15|   413|
|POLYGON ((-73.799...|      16|   483|
|POLYGON ((-73.895...|      17|   208|
|POLYGON ((-73.878...|      18|   226|
|POLYGON ((-73.991...|      19|   312|
|POLYGON ((-73.959...|      20|   314|
+--------------------+--------+------+
only showing top 20 rows



In [9]:
nyc_districts.createOrReplaceTempView("nyc_districts")

In [10]:
crime_nyc = spark.sql("""
    with crimes as (
                    select *
            ,ST_Point(Longitude , Latitude) as geom
        from nyc_crime_years
        where Latitude is not null or Longitude is not null
        and year(to_date(CMPLNT_FR_DT, 'MM/dd/yyyy')) != 1025
    ),
    districs as (
    select 
        OBJECTID
        ,BoroCD as district
        ,geometry as geom
    from nyc_districts
    )
                      select
                        crimes.*
                      , districs.district
                        , ST_Intersects(crimes.geom, districs.geom) as intersects
                      from crimes
                      join districs on ST_Intersects(crimes.geom, districs.geom)
""")

crime_nyc.write.mode('overwrite').parquet(f"{path_data}/crime_nyc_years.parquet")
crime_nyc.show(5)

+----------+-----------+-------------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+----------------+-----+-----------+-----------------+--------------------+--------+--------------------+-----+--------------------+--------------------+----------+------------+--------------+---------+--------+----------------+-------------+--------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+--------------------+--------+----------+
|CMPLNT_NUM|ADDR_PCT_CD|      BORO_NM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|CRM_ATPT_CPTD_CD|HADEVELOPT|HOUSING_PSA|JURISDICTION_CODE|      JURIS_DESC|KY_CD| LAW_CAT_CD|LOC_OF_OCCUR_DESC|           OFNS_DESC|PARKS_NM|         PATROL_BORO|PD_CD|             PD_DESC|       PREM_TYP_DESC|    RPT_DT|STATION_NAME|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|X_COORD_CD|Y_COORD_CD|       

In [11]:
crime_nyc.select("District").distinct().count()

71

In [13]:
crime_nyc = spark.read.parquet(f"{path_data}/crime_nyc_years.parquet")
crime_nyc.show(5)

+----------+-----------+---------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+-------------------+-----+----------+-----------------+--------------------+--------+--------------------+-----+--------------------+--------------------+----------+------------+--------------+---------+--------+----------------+-------------+--------------------+-------+----------+----------+-----------------+------------------+--------------------+------------------------+--------------------+--------+----------+
|CMPLNT_NUM|ADDR_PCT_CD|  BORO_NM|CMPLNT_FR_DT|       CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|CRM_ATPT_CPTD_CD|HADEVELOPT|HOUSING_PSA|JURISDICTION_CODE|         JURIS_DESC|KY_CD|LAW_CAT_CD|LOC_OF_OCCUR_DESC|           OFNS_DESC|PARKS_NM|         PATROL_BORO|PD_CD|             PD_DESC|       PREM_TYP_DESC|    RPT_DT|STATION_NAME|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|X_COORD

In [14]:
crime_nyc.createOrReplaceTempView("crime_nyc_years")

In [16]:
df_px = spark.read.csv(f"{path_data}/rgb_districtsnyc.csv", header=True, inferSchema=True)
df_px.show()

+---+---+---+--------+
|  R|  G|  B|District|
+---+---+---+--------+
|254|255|255|     201|
|255|254|255|     201|
|253|255|254|     201|
|255|255|253|     201|
|255|255|253|     201|
|254|254|255|     201|
|254|255|255|     201|
|255|255|253|     201|
|255|254|255|     201|
|254|255|255|     201|
|255|253|254|     201|
|255|255|251|     201|
|254|255|255|     201|
|254|255|255|     201|
|254|255|253|     201|
|255|254|255|     201|
|255|254|252|     201|
|254|255|255|     201|
|254|255|255|     201|
|253|254|255|     201|
+---+---+---+--------+
only showing top 20 rows



In [17]:
aggreg_df = df_px.groupby("District").agg(
    F.avg("R").alias("R_avg"),
    F.stddev("R").alias("R_stddev"),
    F.expr("percentile_approx(R, 0.5)").alias("R_median"),
    F.expr("percentile_approx(R, 0.25)").alias("R_q25"),
    F.expr("percentile_approx(R, 0.75)").alias("R_q75"),
    
    F.avg("G").alias("G_avg"),
    F.stddev("G").alias("G_stddev"),
    F.expr("percentile_approx(G, 0.5)").alias("G_median"),
    F.expr("percentile_approx(G, 0.25)").alias("G_q25"),
    F.expr("percentile_approx(G, 0.75)").alias("G_q75"),
    
    F.avg("B").alias("B_avg"),
    F.stddev("B").alias("B_stddev"),
    F.expr("percentile_approx(B, 0.5)").alias("B_median"),
    F.expr("percentile_approx(B, 0.25)").alias("B_q25"),
    F.expr("percentile_approx(B, 0.75)").alias("B_q75"),
    F.avg( (F.col("R") + F.col("G") + F.col("B")) / 3 ).alias("brightness_avg"),
    F.avg( F.when( (F.col("R")+F.col("G")+F.col("B"))/3 < 50, 1).otherwise(0) ).alias("dark_pixel_ratio"),

)
aggreg_df.show()

+--------+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+--------------------+
|District|             R_avg|          R_stddev|R_median|R_q25|R_q75|             G_avg|          G_stddev|G_median|G_q25|G_q75|             B_avg|          B_stddev|B_median|B_q25|B_q75|    brightness_avg|    dark_pixel_ratio|
+--------+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+------------------+--------+-----+-----+------------------+--------------------+
|     211| 152.5937747819191| 56.98654506605884|     135|  107|  191|125.04837430610627| 68.84321103108333|     100|   71|  165| 78.88996827914353| 86.02875245369897|      39|   18|  109|118.84403912238967| 0.06641554321966693|
|     210|169.54456951627623|  64.3848252890404|     157|  113|  250|146.81959233343474|

In [18]:
aggreg_df.createOrReplaceTempView("aggreg_df")

nyc = spark.sql("""
        select     
            a.R_avg
            ,a.G_avg
            ,a.B_avg    
            ,a.R_stddev
            ,a.G_stddev
            ,a.B_stddev
            ,a.R_median
            ,a.G_median
            ,a.B_median
            ,a.R_q25
            ,a.G_q25
            ,a.B_q25
            ,a.R_q75
            ,a.G_q75
            ,a.B_q75
            ,a.brightness_avg
            ,a.dark_pixel_ratio            
            ,cr.*
        from aggreg_df a        
        JOIN crime_nyc_years cr
        on a.District = cr.District
""")

nyc.show(5)

+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+----------+-----------+-------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+----------------+-----+-----------+-----------------+--------------------+--------+-----------------+-----+--------------------+--------------------+----------+------------+--------------+--------------+--------+----------------+-------------+--------------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+--------------------+--------+----------+
|            R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_median|G_median|B_median|R_q25|G_q25|B_q25|R_q75|G_q75|B_q75|    brightness_avg|   dark_pixel_ratio|CMPLNT_NUM|ADDR_P

In [19]:
nyc.select("OFNS_DESC").distinct().show(100, truncate=False)

+--------------------------------+
|OFNS_DESC                       |
+--------------------------------+
|ANTICIPATORY OFFENSES           |
|CHILD ABANDONMENT/NON SUPPORT 1 |
|NEW YORK CITY HEALTH CODE       |
|VEHICLE AND TRAFFIC LAWS        |
|KIDNAPPING & RELATED OFFENSES   |
|PETIT LARCENY OF MOTOR VEHICLE  |
|OFF. AGNST PUB ORD SENSBLTY &   |
|FELONY ASSAULT                  |
|ALCOHOLIC BEVERAGE CONTROL LAW  |
|OFFENSES RELATED TO CHILDREN    |
|CRIMINAL MISCHIEF & RELATED OF  |
|THEFT-FRAUD                     |
|JOSTLING                        |
|MISCELLANEOUS PENAL LAW         |
|ARSON                           |
|OFFENSES AGAINST THE PERSON     |
|GAMBLING                        |
|SEX CRIMES                      |
|HARRASSMENT 2                   |
|CRIMINAL TRESPASS               |
|BURGLAR'S TOOLS                 |
|OFFENSES INVOLVING FRAUD        |
|ASSAULT 3 & RELATED OFFENSES    |
|INTOXICATED/IMPAIRED DRIVING    |
|INTOXICATED & IMPAIRED DRIVING  |
|FRAUDS             

In [20]:
df = nyc.withColumn("CMPLNT_FR_DT_PARSED", to_date("CMPLNT_FR_DT", "MM/dd/yyyy")) \
                 .withColumn("CRIME_YEAR", year("CMPLNT_FR_DT_PARSED")) \
                 .withColumn("CRIME_MONTH", month("CMPLNT_FR_DT_PARSED")) \
                 .withColumn("CRIME_DAY", dayofmonth("CMPLNT_FR_DT_PARSED"))

agg_df = df.groupBy("district", "CRIME_YEAR", "CRIME_MONTH") \
                    .agg(F.count("*").alias("total_crimes_dist")) \
                    .orderBy("district", "CRIME_YEAR", "CRIME_MONTH")

df = df.join(agg_df, on=["district", "CRIME_YEAR", "CRIME_MONTH"], how="left")

df.show()


+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+----------+-----------+-------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+-------------------+-----+-----------+-----------------+--------------------+--------+-----------------+-----+--------------------+--------------------+----------+------------+--------------+--------------+--------+----------------+-------------+--------------------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+--------------------+----------+-------------------+---------+-----------------+
|district|CRIME_YEAR|CRIME_MONTH|            R_avg|             G_avg|            B_avg|         R_stddev|         G_stddev|         B_stddev|R_media

In [21]:
agg_race_vic_df = df.groupBy("District", "CRIME_YEAR","CRIME_MONTH" ).agg(
    avg(when(df.VIC_RACE == 'WHITE', 1).otherwise(0)).alias("mean_white"),
    avg(when(df.VIC_RACE == 'BLACK', 1).otherwise(0)).alias("mean_black"),
    avg(when(df.VIC_RACE == 'BLACK HISPANIC', 1).otherwise(0)).alias("mean_black_hispanic"),
    avg(when(df.VIC_RACE == 'WHITE HISPANIC', 1).otherwise(0)).alias("mean_hispanic"),
    avg(when(df.VIC_RACE == 'AMERICAN INDIAN/ALASKAN NATIVE', 1).otherwise(0)).alias("mean_amer_indian"),
    avg(when(df.VIC_RACE == 'ASIAN / PACIFIC ISLANDER', 1).otherwise(0)).alias("mean_asian")

).orderBy("District")
df = df.join(agg_race_vic_df, on=["District","CRIME_YEAR","CRIME_MONTH"], how="left")

In [22]:
from pyspark.sql.functions import when, avg, countDistinct, count, col, row_number

agg_age_df = df.groupBy("District", "CRIME_YEAR","CRIME_MONTH").agg(
    avg(when(df.SUSP_AGE_GROUP == '18-24', 1/7).otherwise(0)).alias("pct_age_18_24"),
    avg(when(df.SUSP_AGE_GROUP == '25-44', 1/20).otherwise(0)).alias("pct_age_25_44"),
    avg(when(df.SUSP_AGE_GROUP == '45-64', 1).otherwise(0)).alias("pct_age_45_64"),
    avg(when(df.SUSP_AGE_GROUP == '<18', 1).otherwise(0)).alias("pct_age_<18"),
    avg(when(df.SUSP_AGE_GROUP == '65+', 1).otherwise(0)).alias("pct_age_65_plus")
)
df = df.join(agg_age_df, on=["District","CRIME_YEAR","CRIME_MONTH"], how="left")

In [23]:
agg_sex_df = df.groupBy("District","CRIME_YEAR","CRIME_MONTH").agg(
    avg(when(df.SUSP_SEX == 'M', 1).otherwise(0)).alias("mean_M"),
    avg(when(df.SUSP_SEX == 'F', 1).otherwise(0)).alias("mean_F"),
    avg(when(df.SUSP_SEX == 'U', 1).otherwise(0)).alias("mean_U")


).orderBy("District")
df = df.join(agg_sex_df, on=["District","CRIME_YEAR","CRIME_MONTH"], how="left")

In [24]:
top_offenses = (
    df.groupBy("OFNS_DESC")
    .count()
    .orderBy("count", ascending=False)
    .limit(15)
    .select("OFNS_DESC")
    .rdd.flatMap(lambda x: x)
    .collect()
)


agg_exprs = [
    avg(when(col("OFNS_DESC") == offense, 1).otherwise(0)).alias(f"pct_{offense.lower().replace(' ', '_')}")
    for offense in top_offenses
]

agg_offense_df = df.groupBy("District","CRIME_YEAR","CRIME_MONTH").agg(*agg_exprs)

agg_offense_df.show(50)

+--------+----------+-----------+-------------------+-------------------+--------------------------------+----------------------------------+--------------------+----------------------------+--------------------+---------------------------+-------------------+-----------------------------------+---------------------------------+----------------------------------+--------------------+--------------------+--------------------+
|District|CRIME_YEAR|CRIME_MONTH|  pct_petit_larceny|  pct_harrassment_2|pct_assault_3_&_related_offenses|pct_criminal_mischief_&_related_of|   pct_grand_larceny|pct_vehicle_and_traffic_laws|  pct_felony_assault|pct_miscellaneous_penal_law|pct_dangerous_drugs|pct_other_offenses_related_to_theft|pct_off._agnst_pub_ord_sensblty_&|pct_grand_larceny_of_motor_vehicle|         pct_robbery|        pct_burglary|      pct_sex_crimes|
+--------+----------+-----------+-------------------+-------------------+--------------------------------+----------------------------------+-

In [25]:
df = df.join(agg_offense_df, on=["District", "CRIME_YEAR", "CRIME_MONTH"], how="left")
df.show(5)

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+----------+-----------+-------+------------+-------------------+------------+------------+----------------+----------+-----------+-----------------+-------------------+-----+-----------+-----------------+--------------------+--------+-----------------+-----+--------------------+--------------------+----------+------------+--------------+--------------+--------+----------------+-------------+--------------+-------+----------+----------+----------------+-----------------+--------------------+------------------------+--------------------+----------+-------------------+---------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------

In [26]:
df = df.drop("HOUSING_PSA","CMPLNT_FR_DT_PARSED","OFNS_DESC","LAW_CAT_CD","CRM_ATPT_CPTD_CD", "CMPLNT_NUM", "PARKS_NM", "SUSP_AGE_GROUP",  "HADEVELOPT","ADDR_PCT_CD","CMPLNT_TO_DT","CMPLNT_FR_TM","CMPLNT_FR_DT","CMPLNT_TO_TM", "RPT_DT",   "JURIS_DESC","PD_DESC",  "KY_CD","PD_CD","STATION_NAME","STATION_NAME",  "PREM_TYP_DESC","TRANSIT_DISTRICT", "JURISDICTION_CODE","LOC_OF_OCCUR_DESC", "Latitude", "Longitude" , "Lat_Lon", "geom",  "PATROL_BORO", "X_COORD_CD", "Y_COORD_CD", "New Georeferenced Column", "intersects" ,"LAW_CODE")
df.show(5)

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+----------------------------------+-------------------+----------------------------+--------------------+---------------------------+--------------------+-----------------------------------+---------------------------------+----------------------------------+--------------------+-------------------+-

In [19]:
df.select("CRIME_YEAR").distinct().show(100)

+----------+
|CRIME_YEAR|
+----------+
|      1025|
|      2025|
|      1975|
|      2003|
|      2018|
|      2015|
|      2023|
|      2006|
|      2022|
|      1925|
|      2013|
|      1997|
|      1994|
|      2014|
|      2019|
|      2004|
|      2020|
|      2012|
|      2009|
|      2016|
|      1995|
|      2001|
|      2024|
|      2005|
|      1984|
|      2000|
|      2010|
|      2011|
|      2008|
|      2017|
|      1999|
|      2002|
|      2021|
|      1993|
+----------+



In [27]:
df.write.mode('overwrite').parquet(f"{path_data}/crime_nyc_years_aggregated.parquet")

### Some other datasets

In [35]:
nyc = spark.read.parquet(f"{path_data}/crime_nyc_years_aggregated.parquet")
nyc.show(5)
nyc.createOrReplaceTempView("nyc")

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+----------------------------------+-------------------+----------------------------+--------------------+---------------------------+--------------------+-----------------------------------+---------------------------------+----------------------------------+--------------------+-------------------+-

In [36]:
nyc_landm = spark.read.csv(f"{path_data}/Individual_Landmark.csv", header=True, inferSchema=True)
nyc_landm.show(5)

+--------------------+-------+----------+-------+-----+---+--------------------+-----+--------+---------+--------------+----------+----------+--------------------+--------------------+---------------+----------+-------+--------------------+---------+---------+----------+----------------+----------------+-------------+---------+--------------------+---------+-------------------+---------+---------+--------------------+--------------------+-------+------+
|            the_geom|    BIN|       BBL|Borough|Block|Lot|          Des_Addres|Circa|Date_Low|Date_High|    Date_Combo|Alt_Date_1|Alt_Date_2|          Arch_Build|           Own_Devel|     Alt_Arch_1|Alt_Arch_2|Altered|          Style_Prim|Style_Sec|Style_Oth|  Mat_Prim|         Mat_Sec|       Mat_Third|     Mat_Four|Mat_Other|            Use_Orig|Use_Other|         Build_Type|Build_Oth|Build_Nme|               Notes|           Hist_Dist|LM_Orig|LM_New|
+--------------------+-------+----------+-------+-----+---+--------------------+----

In [37]:
nyc_landm = nyc_landm.drop("Block","BBL", "BIN",  "Lot", "Arch_Build","Own_Devel","Alt_Arch_1","Mat_Third",  "Alt_Arch_2", "Mat_Other","Build_Oth",    "Date_Combo", "Circa" , "Alt_Date_1", "Alt_Date_2", "Date_High","Altered", "Style_Oth", "Mat_Sec","Mat_Four","LM_Orig","LM_New",   "Use_Other", "Build_Nme" , "Notes" ,   "Style_Sec" )
nyc_landm.show(5)
nyc_landm.createOrReplaceTempView("land")

+--------------------+-------+--------------------+--------+--------------------+----------+--------------------+-------------------+--------------------+
|            the_geom|Borough|          Des_Addres|Date_Low|          Style_Prim|  Mat_Prim|            Use_Orig|         Build_Type|           Hist_Dist|
+--------------------+-------+--------------------+--------+--------------------+----------+--------------------+-------------------+--------------------+
|MULTIPOLYGON (((-...|     BK|803 East 17th Street|    1912|      Not determined|Wood Frame| Outbuilding, garage|             Garage|Fiske Terrace-Mid...|
|MULTIPOLYGON (((-...|     BK|814 East 18th Str...|       0|      Not determined|     Brick| Outbuilding, garage|             Garage|Fiske Terrace-Mid...|
|MULTIPOLYGON (((-...|     BK|  1653 Glenwood Road|    1906|    Colonial Revival|Wood Frame|Residential, sing...|Free-standing House|Fiske Terrace-Mid...|
|MULTIPOLYGON (((-...|     BK| 54 Wellington Court|    1911|       Art

In [38]:
nyc_districts = (
    spark.read.format("geojson").option("multiLine", "true").load(f"{path_data}/nyc_districts.json")
    .selectExpr("explode(features) as features")
    .select("features.*")
    .withColumn("OBJECTID", expr("properties['OBJECTID']"))
    .withColumn("BoroCD", expr("properties['BoroCD']"))
    .drop("properties")
    .drop("type")
    .drop("id")
)
nyc_districts.show(truncate=True)
nyc_districts.createOrReplaceTempView("nyc_districts")

+--------------------+--------+------+
|            geometry|OBJECTID|BoroCD|
+--------------------+--------+------+
|POLYGON ((-73.970...|       1|   308|
|POLYGON ((-73.801...|       2|   410|
|POLYGON ((-73.881...|       3|   206|
|POLYGON ((-73.867...|       4|   480|
|POLYGON ((-73.984...|       5|   104|
|POLYGON ((-73.980...|       6|   107|
|POLYGON ((-74.193...|       7|   503|
|MULTIPOLYGON (((-...|       8|   595|
|POLYGON ((-73.877...|       9|   207|
|POLYGON ((-73.925...|      10|   317|
|POLYGON ((-73.840...|      11|   481|
|POLYGON ((-73.872...|      12|   227|
|POLYGON ((-73.820...|      13|   409|
|POLYGON ((-73.903...|      14|   316|
|POLYGON ((-73.728...|      15|   413|
|POLYGON ((-73.799...|      16|   483|
|POLYGON ((-73.895...|      17|   208|
|POLYGON ((-73.878...|      18|   226|
|POLYGON ((-73.991...|      19|   312|
|POLYGON ((-73.959...|      20|   314|
+--------------------+--------+------+
only showing top 20 rows



In [40]:
land = spark.sql("""
WITH
  valid_land AS (
    SELECT *
    FROM land
    WHERE the_geom RLIKE '^(POINT|LINESTRING|POLYGON|MULTI)'
  ),

  geom_land AS (
    SELECT *,
           ST_GeomFromWKT(the_geom) AS geom
    FROM valid_land
  ),

  geom_dist AS (
    SELECT
      BoroCD   AS district,
      geometry AS geom
    FROM nyc_districts
  )

SELECT
  l.*,
  d.district
FROM geom_land l
JOIN geom_dist d
  ON ST_Intersects(l.geom, d.geom)
""")

land.write.mode("overwrite") \
   .parquet(f"{path_data}/landmark_distr.parquet")

land.show(5)


+--------------------+-------+--------------------+--------+--------------------+----------+--------------------+-------------------+--------------------+--------------------+--------+
|            the_geom|Borough|          Des_Addres|Date_Low|          Style_Prim|  Mat_Prim|            Use_Orig|         Build_Type|           Hist_Dist|                geom|district|
+--------------------+-------+--------------------+--------+--------------------+----------+--------------------+-------------------+--------------------+--------------------+--------+
|MULTIPOLYGON (((-...|     BK|803 East 17th Street|    1912|      Not determined|Wood Frame| Outbuilding, garage|             Garage|Fiske Terrace-Mid...|MULTIPOLYGON (((-...|     314|
|MULTIPOLYGON (((-...|     BK|814 East 18th Str...|       0|      Not determined|     Brick| Outbuilding, garage|             Garage|Fiske Terrace-Mid...|MULTIPOLYGON (((-...|     314|
|MULTIPOLYGON (((-...|     BK|  1653 Glenwood Road|    1906|    Colonial Re

In [41]:
land = land.drop("geom", "the_geom", "Borough","Des_Addres")

In [25]:
nyc.select("district").distinct().count()

66

In [5]:
nyc_landm.select("Style_Prim").distinct().show(100)

+--------------------+
|          Style_Prim|
+--------------------+
|Greek Revival (late)|
| White City Movement|
|Romanesque Reviva...|
|Dutch Renaissance...|
|Dutch Colonial Re...|
|Classicism (simpl...|
|             Federal|
| Post-War Vernacular|
|   Collegiate Gothic|
|Altered Romanesqu...|
|   French Beaux-Arts|
|    Eclectic-Diverse|
|Victorian Gothic ...|
|                None|
|Second Empire wit...|
|Queen Anne with a...|
|Transitional Gree...|
|Vernacular French...|
|    Victorian Gothic|
|    Daylight Factory|
|Industrial Neocla...|
|          Vernacular|
|Spanish Colonial ...|
|        neo-Colonial|
|Altered Dutch Ren...|
|Renaissance Reviv...|
|Classical (simpli...|
|      High Victorian|
|            Medieval|
|Italianate with a...|
|          Romanesque|
|neo-Italian Renai...|
|French Renaissanc...|
|            Stripped|
|Modified Italiana...|
|Transitional Fren...|
|       Greek Revival|
|   Stripped neo-Grec|
|Modified Spanish ...|
| English Garden City|
|    Altere

In [42]:
from pyspark.sql.functions import col, when, avg, desc

top_styles = (
    land
      .groupBy("Style_Prim")
      .count()
      .orderBy(desc("count"))
      .limit(15)
      .select("Style_Prim")
      .rdd
      .flatMap(lambda row: row)
      .collect()
)

agg_exprs = [
    avg(
      when(col("Style_Prim") == style, 1)
      .otherwise(0)
    ).alias(f"pct_{style.lower().replace(' ', '_')}")
    for style in top_styles
]

agg_style_df = (
    land
      .groupBy("district")
      .agg(*agg_exprs)
)

agg_style_df.show(50, truncate=False)

+--------+---------------------+-----------------------+--------------------+--------------------+----------------------+---------------------+---------------------+---------------------+---------------------+-------------------------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|district|pct_not_determined   |pct_renaissance_revival|pct_neo-grec        |pct_italianate      |pct_romanesque_revival|pct_greek_revival    |pct_queen_anne       |pct_colonial_revival |pct_neo-renaissance  |pct_simplified_colonial_revival_or_art_deco|pct_beaux-arts       |pct_none             |pct_neo-georgian     |pct_neoclassical     |pct_anglo-italianate |
+--------+---------------------+-----------------------+--------------------+--------------------+----------------------+---------------------+---------------------+---------------------+---------------------+-------------------------------------------+-------------------

In [43]:
df = nyc.join(agg_style_df, on=["District"], how="left")
df.show(5)

+--------+----------+-----------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+--------+--------+--------+-----+-----+-----+-----+-----+-----+------------------+-------------------+-------+--------------+--------+-------------+--------------+-------+---------+-----------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+----------------------------------+-------------------+----------------------------+--------------------+---------------------------+--------------------+-----------------------------------+---------------------------------+----------------------------------+--------------------+-------------------+-

42